In [219]:
'''Solve the Profile HMM Problem using class object'''

import pandas as pd
import numpy as np

#file='input.txt'
file='input2.txt'
#file='../../Downloads/dataset_11632_2.txt'
#file='../../Downloads/rosalind_ba10e.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

th=float(lines[0])
alph=map(str,lines[2].split('\t'))
aln=[list(line) for line in lines[4:]]
print th,alph,aln


class HMM:
    import pandas as pd
    import numpy as np

    def __init__(self, threshold, alphabet, alignment):
        self.th = threshold #e.g. 0.301
        self.alph = alphabet #e.g. ['A', 'B', 'C', 'D', 'E']
        self.aln = alignment #e.g.  [['E', 'B', 'A'], ['E', '-', 'D'], ['E', 'B', '-'], ['E', 'E', 'D'], ['E', 'B', 'D'], ['E', 'B', 'E'], ['E', '-', 'D'], ['E', '-', 'D']]
        self.I = self.find_insertions()[1]
        self.good_col = self.find_insertions()[0]
        self.hmm = ['S','I0']+[St+str(n) for n in range(1,self.good_col+1) for St in ['M','D','I']]+['E']
    
    def find_insertions(self):
        a=pd.DataFrame(self.aln)
        I={}
        good_col=len(a.columns)
        M_counter=0
        for n in range(len(a.columns)):
            col=n
            try:
                if float(a[col].value_counts().to_dict()['-'])/sum(a[col].value_counts())>th:
                    try:
                        assert I[n-1]
                        I[n]=I[n-1]
                    except KeyError:
                        I[n]='I'+str(M_counter)
                    good_col-=1
                else:
                    M_counter+=1
            except KeyError:
                M_counter+=1
        return good_col,I
    
    def transition_matrix(self):
        Trans=pd.DataFrame(0,index=self.hmm,columns=self.hmm)
        I=self.I
        aln=self.aln
        #print Trans
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            path=['S']
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        #good_counter+=1
                        continue
                    else:
                        if path[-1][0]=='I':
                            path+=[path[-1]]
                            #print path,i,good_counter
                            Trans[path[-1]][path[-2]]+=1
                            continue
                        else:
                            path+=['I'+str(M_counter-1)]
                            #print path,i,good_counter
                            Trans[path[-1]][path[-2]]+=1
                            #good_counter+=1
                elif seq[i] =='-':
                    path+=['D'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
                else:
                    path+=['M'+str(M_counter)]
                    #print path,i
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
            path+=['E']
            Trans[path[-1]][path[-2]]+=1
        for row in Trans.index:
             Trans.loc[row]=Trans.loc[row]/Trans.loc[row].sum()
        Trans=Trans.fillna(0)
        return Trans 

    def emission_matrix(self):
        Em=pd.DataFrame(0,index=self.hmm,columns=self.alph)
        I=self.I
        aln=self.aln
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        Em[seq[i]][I[i]]+=1
                elif seq[i]=='-':
                    M_counter+=1
                    pass
                else:
                    Em[seq[i]]['M'+str(M_counter)]+=1
                    M_counter+=1
        for row in Em.index:
             Em.loc[row]=Em.loc[row]/Em.loc[row].sum()
        Em=Em.fillna(0)
        return Em  
        

#print HMM(th,alph,aln).find_insertions()

print HMM(th,alph,aln).transition_matrix()
print '--------'
print HMM(th,alph,aln).emission_matrix()

HMM(th,alph,aln).transition_matrix().to_csv('output.txt', mode='w', header=True,sep='\t')
with open('output.txt', 'a') as fa:
             fa.write('--------'+'\n')
HMM(th,alph,aln).emission_matrix().to_csv('output.txt', mode='a', header=True,sep='\t')

ValueError: invalid literal for float(): 0.399	0.01

In [220]:
'''Solve the Profile HMM with Pseudocounts Problem'''
import pandas as pd
import numpy as np

#file='input.txt'
file='input2.txt'
#file='../../Downloads/dataset_11632_4.txt'
#file='../../Downloads/rosalind_ba10f.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

print map(str,lines[0].split('\t'))[-1]
th=float(map(str,lines[0].split('\t'))[0])
pseudo=float(map(str,lines[0].split('\t'))[-1])
alph=map(str,lines[2].split('\t'))
aln=[list(line) for line in lines[4:]]
print th,pseudo,alph,aln


class HMM_pseudo:
    import pandas as pd
    import numpy as np

    def __init__(self, threshold, pseudo, alphabet, alignment):
        self.th = threshold #e.g. 0.301
        self.pseudo = pseudo # e.g. 0.01
        self.alph = alphabet #e.g. ['A', 'B', 'C', 'D', 'E']
        self.aln = alignment #e.g.  [['E', 'B', 'A'], ['E', '-', 'D'], ['E', 'B', '-'], ['E', 'E', 'D'], ['E', 'B', 'D'], ['E', 'B', 'E'], ['E', '-', 'D'], ['E', '-', 'D']]
        self.I = self.find_insertions()[1]
        self.good_col = self.find_insertions()[0]
        self.hmm = ['S','I0']+[St+str(n) for n in range(1,self.good_col+1) for St in ['M','D','I']]+['E']
    
    def find_insertions(self):
        a=pd.DataFrame(self.aln)
        I={}
        good_col=len(a.columns)
        M_counter=0
        for n in range(len(a.columns)):
            col=n
            try:
                if float(a[col].value_counts().to_dict()['-'])/sum(a[col].value_counts())>th:
                    try:
                        assert I[n-1]
                        I[n]=I[n-1]
                    except KeyError:
                        I[n]='I'+str(M_counter)
                    good_col-=1
                else:
                    M_counter+=1
            except KeyError:
                M_counter+=1
        return good_col,I
    
    def transition_matrix(self):
        Trans=pd.DataFrame(0,index=self.hmm,columns=self.hmm)
        pseudo=self.pseudo
        I=self.I
        aln=self.aln
        good_col=self.good_col
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            path=['S']
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        if path[-1][0]=='I':
                            path+=[path[-1]]
                            Trans[path[-1]][path[-2]]+=1
                            continue
                        else:
                            path+=['I'+str(M_counter-1)]
                            Trans[path[-1]][path[-2]]+=1
                elif seq[i] =='-':
                    path+=['D'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
                else:
                    path+=['M'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
            path+=['E']
            Trans[path[-1]][path[-2]]+=1
        
        ### ger rel, add pseudocounts and get rel ###
        ## start
        subset=Trans.loc['S':'I0', 'I0':'D1']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
        Trans.loc['S':'I0', 'I0':'D1']=subset
        ## between matches
        for n in range(1,good_col):
            subset=Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]
            for row in subset.index:
                subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
                subset.loc[row]=subset.loc[row].fillna(0)
                subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
            Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]=subset
        ## end
        subset=Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+2*pseudo)
        Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']=subset
        return Trans
    
    def emission_matrix(self):
        Em=pd.DataFrame(0,index=self.hmm,columns=self.alph)
        I=self.I
        aln=self.aln
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        Em[seq[i]][I[i]]+=1
                elif seq[i]=='-':
                    M_counter+=1
                    pass
                else:
                    Em[seq[i]]['M'+str(M_counter)]+=1
                    M_counter+=1
        for row in Em.index:
            if row[0] in ['S','D','E']:
                pass
            else:
                Em.loc[row]=Em.loc[row]/Em.loc[row].sum()
                Em.loc[row]=Em.loc[row].fillna(0)
                Em.loc[row]=(Em.loc[row]+pseudo)/(Em.loc[row].sum()+len(self.alph)*pseudo)
        return Em  


#print HMM(th,alph,aln).find_insertions()

print HMM_pseudo(th,pseudo,alph,aln).transition_matrix()
#print '--------'
#print HMM(th,alph,aln).emission_matrix()

HMM_pseudo(th,pseudo,alph,aln).transition_matrix().to_csv('output.txt', mode='w', header=True,sep='\t',float_format='%.3f')
with open('output.txt', 'a') as fa:
             fa.write('--------'+'\n')
HMM_pseudo(th,pseudo,alph,aln).emission_matrix().to_csv('output.txt', mode='a', header=True,sep='\t',float_format='%.3f')

0.01
0.399 0.01 ['A B C D E'] [['E', 'D', '-', 'B', 'C', 'B', 'D', 'A', 'C'], ['-', 'D', '-', 'A', 'B', 'B', 'D', 'A', 'C'], ['E', 'D', '-', '-', 'E', 'B', 'D', '-', 'C'], ['-', 'C', '-', 'B', 'C', 'B', '-', 'D', '-'], ['A', 'D', '-', 'B', 'C', '-', 'C', 'A', '-'], ['-', 'D', 'D', 'B', '-', 'B', 'A', '-', 'C']]
    S        I0        M1        D1        I1        M2        D2        I2  \
S   0  0.495146  0.495146  0.009709  0.000000  0.000000  0.000000  0.000000   
I0  0  0.009709  0.980583  0.009709  0.000000  0.000000  0.000000  0.000000   
M1  0  0.000000  0.000000  0.000000  0.171521  0.656958  0.171521  0.000000   
D1  0  0.000000  0.000000  0.000000  0.333333  0.333333  0.333333  0.000000   
I1  0  0.000000  0.000000  0.000000  0.009709  0.980583  0.009709  0.000000   
M2  0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.009709   
D2  0  0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.009709   
I2  0  0.000000  0.000000  0.000000  0.000000  0.000000

KeyError: 'E'

In [17]:
'''Solve the Sequence Alignment with Profile HMM Problem'''

class HMM_pseudo:
    import pandas as pd
    import numpy as np

    def __init__(self, threshold, pseudo, alphabet, alignment):
        self.th = threshold #e.g. 0.301
        self.pseudo = pseudo # e.g. 0.01
        self.alph = alphabet #e.g. ['A', 'B', 'C', 'D', 'E']
        self.aln = alignment #e.g.  [['E', 'B', 'A'], ['E', '-', 'D'], ['E', 'B', '-'], ['E', 'E', 'D'], ['E', 'B', 'D'], ['E', 'B', 'E'], ['E', '-', 'D'], ['E', '-', 'D']]
        self.I = self.find_insertions()[1]
        self.good_col = self.find_insertions()[0]
        self.hmm = ['S','I0']+[St+str(n) for n in range(1,self.good_col+1) for St in ['M','D','I']]+['E']
    
    def find_insertions(self):
        a=pd.DataFrame(self.aln)
        I={}
        good_col=len(a.columns)
        M_counter=0
        for n in range(len(a.columns)):
            col=n
            try:
                if float(a[col].value_counts().to_dict()['-'])/sum(a[col].value_counts())>=th:
                    try:
                        assert I[n-1]
                        I[n]=I[n-1]
                    except KeyError:
                        I[n]='I'+str(M_counter)
                    good_col-=1
                else:
                    M_counter+=1
            except KeyError:
                M_counter+=1
        return good_col,I
    
    def transition_matrix(self):
        Trans=pd.DataFrame(0,index=self.hmm,columns=self.hmm)
        pseudo=self.pseudo
        I=self.I
        aln=self.aln
        good_col=self.good_col
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            path=['S']
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        if path[-1][0]=='I':
                            path+=[path[-1]]
                            Trans[path[-1]][path[-2]]+=1
                            continue
                        else:
                            path+=['I'+str(M_counter-1)]
                            Trans[path[-1]][path[-2]]+=1
                elif seq[i] =='-':
                    path+=['D'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
                else:
                    path+=['M'+str(M_counter)]
                    Trans[path[-1]][path[-2]]+=1
                    M_counter+=1
            path+=['E']
            Trans[path[-1]][path[-2]]+=1
        
        ### ger rel, add pseudocounts and get rel ###
        ## start
        subset=Trans.loc['S':'I0', 'I0':'D1']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
        Trans.loc['S':'I0', 'I0':'D1']=subset
        ## between matches
        for n in range(1,good_col):
            subset=Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]
            for row in subset.index:
                subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
                #print 'subset.loc[row]', row
                #print subset.loc[row]
                subset.loc[row]=subset.loc[row].fillna(0)
                subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+3*pseudo)
            Trans.loc['M'+str(n):'I'+str(n), 'I'+str(n):'D'+str(n+1)]=subset
        ## end
        subset=Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']
        for row in subset.index:
            subset.loc[row]=subset.loc[row]/subset.loc[row].sum()
            subset.loc[row]=subset.loc[row].fillna(0)
            subset.loc[row]=(subset.loc[row]+pseudo)/(subset.loc[row].sum()+2*pseudo)
        Trans.loc['M'+str(good_col):'I'+str(good_col), 'I'+str(good_col):'E']=subset
        return Trans
    
    def emission_matrix(self):
        Em=pd.DataFrame(0,index=self.hmm,columns=self.alph)
        I=self.I
        aln=self.aln
        for n in range(len(aln)):
            seq=aln[n]
            M_counter=1
            for i in range(len(seq)):
                if i in I.keys():
                    if seq[i]=='-':
                        continue
                    else:
                        Em[seq[i]][I[i]]+=1
                elif seq[i]=='-':
                    M_counter+=1
                    pass
                else:
                    Em[seq[i]]['M'+str(M_counter)]+=1
                    M_counter+=1
        for row in Em.index:
            if row[0] in ['S','D','E']:
                pass
            else:
                Em.loc[row]=Em.loc[row]/Em.loc[row].sum()
                Em.loc[row]=Em.loc[row].fillna(0)
                Em.loc[row]=(Em.loc[row]+pseudo)/(Em.loc[row].sum()+len(self.alph)*pseudo)
        return Em
    
    #HMM_pseudo(th,pseudo,alph,aln).transition_matrix().to_csv('output.txt', mode='w', header=True,sep='\t')
    def Align_to_HMMprofile(self,x):
        import pandas as pd
        import numpy as np

        #x=self.x
        gc=self.good_col
        Trans=HMM_pseudo(self.th,self.pseudo,self.alph,self.aln).transition_matrix()
        Trans=np.log10(Trans)

        Em=HMM_pseudo(self.th,self.pseudo,self.alph,self.aln).emission_matrix()
        Em=np.log10(Em)
        hmm=self.hmm

        V=pd.DataFrame(-float('Inf'),index=hmm[1:-1],columns=range(len(x)))
        Backtrack=pd.DataFrame(list,index=hmm[1:-1],columns=range(len(x)))
        Index=['S']+['D'+str(n+1) for n in range(gc)]
        sdic={}
        Silent={} #Score for 0th column that contains only silent states
        Silent['S']=0
        for n in range(len(Index)):
            sdic[n]=Index[n]
            if n>0:
                Silent[Index[n]]=Silent[Index[n-1]]+Trans[Index[n]][Index[n-1]]
        ### Fill first column of Viterbi graph
        for node in V.index:
            idx=int(node[1:])
            if node[0]=='D':
                if node=='D1':
                    V[0][node]=V[0]['I0']+Trans[node]['I0']
                    Backtrack[0][node]=[-1,'I0']
                else:
                    V[0][node]=max([V[0][ST+str(idx-1)]+Trans[node][ST+str(idx-1)] for ST in ['M','D','I']])
                    Backtrack[0][node]=[0,[ST+str(idx-1) for ST in ['M','D','I'] if V[0][node]==V[0][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]][0]]
            elif node=='I0':
                V[0][node]=Silent['S']+Trans[node]['S']+Em[x[0]][node]
                Backtrack[0][node]=[-1,'S']
            else:
                V[0][node]=Silent[sdic[idx-1]]+Trans[node][sdic[idx-1]]+Em[x[0]][node]
                Backtrack[0][node]=[-1,sdic[idx-1]]

        ### Fill all following columns of Viterbi graph
        for n in V.columns[1:]:
            for node in V.index:
                idx=int(node[1:])
                if node=='I0':
                    V[n][node]=V[n-1][node]+Trans[node][node]+Em[x[n]][node]
                    Backtrack[n][node]=[int(n-1),node]
                elif node=='M1':
                    V[n][node]=V[n-1]['I0']+Trans[node]['I0']+Em[x[n]][node]
                    Backtrack[n][node]=[n-1,'I0']
                elif node=='D1':
                    V[n][node]=V[n]['I0']+Trans[node]['I0']
                    Backtrack[n][node]=[n,'I0']
                elif node[0]=='I':
                    V[n][node]=max([V[n-1][ST+str(idx)]+Trans[node][ST+str(idx)]+Em[x[n]][node] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n-1,[ST+str(idx) for ST in ['M','D','I'] if V[n][node]==V[n-1][ST+str(idx)]+Trans[node][ST+str(idx)]+Em[x[n]][node]][0]]
                elif node[0]=='M':
                    V[n][node]=max([V[n-1][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]+Em[x[n]][node] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n-1,[ST+str(idx-1) for ST in ['M','D','I'] if V[n][node]==V[n-1][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]+Em[x[n]][node]][0]]
                elif node[0]=='D':
                    V[n][node]=max([V[n][ST+str(idx-1)]+Trans[node][ST+str(idx-1)] for ST in ['M','D','I']])
                    Backtrack[n][node]=[n,[ST+str(idx-1) for ST in ['M','D','I'] if V[n][node]==V[n][ST+str(idx-1)]+Trans[node][ST+str(idx-1)]][0]]

        ### Backtrack
        result=[]
        maxscore=max([V[n][ST]+Trans['E'][ST] for ST in V.index[-3:]])
        node=[ST for ST in V.index[-3:] if V[n][ST]+Trans['E'][ST]==maxscore][0]
        while True:
            prevnode=Backtrack[n][node][1]
            prevcol=Backtrack[n][node][0]
            result+=[node]
            node=prevnode
            n=prevcol
            if node in ['I0','S']:
                break
        return result[::-1]

import pandas as pd
import numpy as np

#file='input.txt'
#file='input2.txt'
#file='../../Downloads/dataset_11632_6.txt'
file='../../Downloads/rosalind_ba10g.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
th=float(map(str,lines[2].split(' '))[0])
pseudo=float(map(str,lines[2].split(' '))[-1])
alph=map(str,lines[4].split(' '))
aln=[list(line) for line in lines[6:]]

#print th,pseudo,alph,aln
print ' '.join(HMM_pseudo(th,pseudo,alph,aln).Align_to_HMMprofile(x))


gc 8
['A', 'E', 'F', 'D', 'F', 'D', 'C']
M4 -0.853970820607
['M3', [-5.1131282214239588, -0.0085158509225296275, -0.11682084638612046], 'D3', [-0.7286341232984398, -0.0085158509225296275, -0.11682084638612046], 'I3', [-5.2894628054361803, -0.47712125471966238, -0.11682084638612046]]
D5 -1.54458875058
M5 -0.979307517916
['M4', [-0.85397082060708984, -0.10435220582652247, -0.020984491482127662], 'D4', [-3.9967438531069188, -0.47712125471966238, -0.020984491482127662], 'I4', [-3.9967438531069188, -0.47712125471966238, -0.020984491482127662]]
D5 -5.37735305552
M6 -2.7450116811
['M5', [-6.1473864156803391, -0.59786387673435426, -0.703086570530851], 'D5', [-5.3773530555188751, -0.0085158509225296275, -0.703086570530851], 'I5', [-1.814417720870972, -0.2275073896944052, -0.703086570530851]]
M1 D2 D3 M4 M5 I5 M6 M7 M8


In [61]:
''' Solve the HMM Parameter Estimation Problem'''

import pandas as pd
import numpy as np
import itertools
import re

file='input.txt'
file='input2.txt'
file='../../Downloads/dataset_11632_8.txt'
file='../../Downloads/rosalind_ba10h.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
alph=map(str,lines[2].split('\t'))
Pi=lines[4]
states=map(str,lines[6].split('\t'))

print x,alph,Pi,states
print

def ParameterEstimation(x,alph,Pi,states)
    Trans=pd.DataFrame(float,columns=states,index=states)
    for combi in itertools.product(states, repeat=2):
        row=combi[0]
        col=combi[1]
        if row not in Pi:
            Trans[col][row]=1./len(states)
        else:
            #print row,col,float(Pi.count(row+col)),Pi[:-1].count(row), len(re.findall('(?='+row+col+')', Pi))
            Trans[col][row]=float(len(re.findall('(?='+row+col+')', Pi)))/Pi[:-1].count(row)
    Em=pd.DataFrame(float,columns=alph,index=states)
    for combi in itertools.product(states,alph):
        row=combi[0]
        col=combi[1]
        if row not in Pi:
            Em[col][row]=1./len(states)
        else:
            Em[col][row]=float(zip(Pi,x).count((row,col)))/Pi.count(row)
    return Trans,Em    

Trans,Em=ParameterEstimation(x,alph,Pi,states)
Trans.to_csv('output.txt', mode='w', header=True,sep='\t',float_format='%.3f')
with open('output.txt', 'a') as fa:
             fa.write('--------'+'\n')
Em.to_csv('output.txt', mode='a', header=True,sep='\t',float_format='%.3f')

['z', 'x', 'z', 'y', 'z', 'z', 'y', 'z', 'y', 'x', 'y', 'x', 'z', 'x', 'z', 'y', 'y', 'y', 'x', 'x', 'z', 'y', 'x', 'x', 'z', 'y', 'x', 'y', 'y', 'y', 'x', 'y', 'z', 'x', 'y', 'y', 'x', 'z', 'z', 'x', 'x', 'x', 'z', 'z', 'y', 'y', 'y', 'y', 'z', 'x', 'z', 'x', 'z', 'z', 'x', 'x', 'y', 'z', 'x', 'z', 'y', 'x', 'y', 'z', 'y', 'x', 'z', 'y', 'x', 'z', 'y', 'x', 'x', 'z', 'x', 'y', 'y', 'z', 'z', 'x', 'y', 'z', 'x', 'x', 'z', 'z', 'x', 'y', 'y', 'z', 'x', 'x', 'y', 'y', 'y', 'z', 'y', 'x', 'x', 'x'] ['x', 'y', 'z'] CACBBDCBBBBDCCBCCABDCDCAACDACDBCBDCABCBDDABDDACADCDDBACADBDBDBDACBADADDDDACCDCAADDAACCDCCBBBBDABBCCB ['A', 'B', 'C', 'D']

            A          B          C          D
A        0.15        0.2        0.4       0.25
B  0.08333333  0.3333333  0.1666667  0.4166667
C   0.2592593  0.2962963  0.2222222  0.2222222
D   0.2857143  0.1785714  0.2857143       0.25
--------
           x          y          z
A        0.4        0.3        0.3
B       0.36        0.2       0.44
C  0.259259

In [26]:
'''Implement Viterbi learning for estimating the parameters of an HMM'''
import pandas as pd
import numpy as np
import itertools
import re





def ViterbiLearning(x,alph,states,Trans,Em):
    
    '''Returns the emission and transition matrices resulting from applying Viterbi learning for j iterations.
    INPUT:
    j= 100
    x= ['z', 'y', 'z', 'x', 'z', 'x', 'x', 'x', 'z', 'z']
    alph= ['x', 'y', 'z']
    states= ['A', 'B']
    OUPUT:
    Trans=     A    B # pandas table
        A  0.5  0.5
        B    0    1
    Em=          x         y         z # pandas table
        A  0.333333  0.333333  0.333333
        B       0.4       0.1       0.5
    '''
    
    def FindJiddenPath(x,alp,st,Trans,Em):
        import numpy as np
        '''
        Finds a path that maximizes the (unconditional) probability Pr(x, Pi) over all possible paths Pi
        Input:
        x= ['z', 'y', 'z', 'x', 'z', 'x', 'x', 'x', 'z', 'z']
        alp= ['x', 'y', 'z']
        st= ['A', 'B']
        Trans=     A    B # pandas table
        A  0.5  0.5
        B    0    1
        Em=          x         y         z # pandas table
            A  0.333333  0.333333  0.333333
            B       0.4       0.1       0.5
        OUTPUT=BBBBBBBBBB
        '''
        Trans=np.log10(Trans)
        Em=np.log10(Em)
        Pi={}
        Backtrack={}
        Pi[0]={}
        [Pi[0].update({st[n]:np.log10(0.5)+Em[x[0]][st[n]]}) for n in range(len(st))]
        for n in range(1,len(x)):
            Pi[n]={}
            [Pi[n].update({st[i]:max([Pi[n-1][st[j]]+Em[x[n]][st[i]]+Trans[st[i]][st[j]] for j in range(len(st))])})\
                           for i in range(len(st))]
            Backtrack[n]={}
            for i in range(len(st)):
                for j in range(len(st)):
                    if Pi[n-1][st[j]]+Em[x[n]][st[i]]+Trans[st[i]][st[j]] == Pi[n][st[i]]:
                        Backtrack[n].update({st[i]:st[j]})
        pi_string=[]
        good_state=[k for k,v in Pi[len(x)-1].items() if v==max(Pi[len(x)-1].values())][0]
        pi_string=good_state
        for n in reversed(range(1,len(x))):
            new_state=Backtrack[n][good_state]
            pi_string+=new_state
            good_state=new_state
        return pi_string[::-1]

    def ParameterEstimation(x,alph,Pi,states):
        '''
        Find optimal parameters explaining the emitted string and hidden path of an HMM
        INPUT:
        x= ['z', 'y', 'z', 'x', 'z', 'x', 'x', 'x', 'z', 'z']
        alph= ['x', 'y', 'z']
        Pi= BBBBBBBBBB
        states= ['A', 'B']

        OUTPUT:
        Trans=     A    B # pandas table
            A  0.5  0.5
            B    0    1
        Em=          x         y         z # pandas table
            A  0.333333  0.333333  0.333333
            B       0.4       0.1       0.5
        '''
        Trans=pd.DataFrame(-float('Inf'),columns=states,index=states)
        for combi in itertools.product(states, repeat=2):
            row=combi[0]
            col=combi[1]
            if row not in Pi:
                Trans[col][row]=1./len(states)
            else:
                Trans[col][row]=float(len(re.findall('(?='+row+col+')', Pi)))/Pi[:-1].count(row)
        Em=pd.DataFrame(-float('Inf'),columns=alph,index=states)
        for combi in itertools.product(states,alph):
            row=combi[0]
            col=combi[1]
            if row not in Pi:
                Em[col][row]=1./len(alph)
            else:
                Em[col][row]=float(zip(Pi,x).count((row,col)))/Pi.count(row)
        return Trans,Em    

    ##########################################
    n=0
    while n<=1:
        Pi=FindJiddenPath(x,alph,states,Trans,Em)
        print 'Pi=',Pi
        Trans,Em=ParameterEstimation(x,alph,Pi,states)
        n+=1
    return Trans,Em



#file='input.txt'
file='input2.txt'
#file='../../Downloads/dataset_11632_10.txt'
#file='../../Downloads/rosalind_ba10i.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

j=int(lines[0])
x=list(lines[2])
alph=map(str,lines[4].split(' '))
states=map(str,lines[6].split(' '))
#print lines[8:]
footer=[len(lines)-n for n in reversed(range(len(lines))) if lines[n][0]=='-'][0]
Trans=pd.read_csv(file, skiprows=8,sep='\t+',header=0,index_col=0,skipfooter=footer)
Em=pd.read_csv(file, skiprows=len(lines)-footer+1,sep='\t+',header=0,index_col=0)
print 'j=',j
print 'x=',x
print 'alph=',alph
print 'states=',states


print Em
#print Em['A']['A']
print Trans
#print Em.columns

Trans,Em=ViterbiLearning(x,alph,states,Trans,Em)
print Trans
print '--------'
print Em

Trans.to_csv('output.txt', mode='w', header=True,sep='\t',float_format='%.3f')
with open('output.txt', 'a') as fa:
             fa.write('--------'+'\n')
Em.to_csv('output.txt', mode='a', header=True,sep='\t',float_format='%.3f')

j= 100
x= ['z', 'y', 'z', 'x', 'z', 'x', 'x', 'x', 'z', 'z']
alph= ['x', 'y', 'z']
states= ['A', 'B']
       x      y      z
A  0.424  0.367  0.209
B  0.262  0.449  0.289
       A      B
A  0.599  0.401
B  0.294  0.706
<module 'numpy' from '/Users/juniper_admin/anaconda/lib/python2.7/site-packages/numpy/__init__.pyc'>
Pi= BBBBBBBBBB
<module 'numpy' from '/Users/juniper_admin/anaconda/lib/python2.7/site-packages/numpy/__init__.pyc'>
Pi= BBBBBBBBBB
     A    B
A  0.5  0.5
B  0.0  1.0
--------
          x         y         z
A  0.333333  0.333333  0.333333
B  0.400000  0.100000  0.500000


/Users/juniper_admin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:129: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/Users/juniper_admin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:130: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
/Users/juniper_admin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:45: RuntimeWarning: divide by zero encountered in log10


In [34]:
'''Solve the Soft Decoding Problem.'''

import pandas as pd
import numpy as np

def FindJiddenPath(x,alp,st,Trans,Em):
    import numpy as np
    '''
    Finds a path that maximizes the (unconditional) probability Pr(x, Pi) over all possible paths Pi
    Input:
    x= ['z', 'y', 'z', 'x', 'z', 'x', 'x', 'x', 'z', 'z']
    alp= ['x', 'y', 'z']
    st= ['A', 'B']
    Trans=     A    B # pandas table
    A  0.5  0.5
    B    0    1
    Em=          x         y         z # pandas table
        A  0.333333  0.333333  0.333333
        B       0.4       0.1       0.5
    OUTPUT=BBBBBBBBBB
    '''
    Trans=np.log10(Trans)
    Em=np.log10(Em)
    Pi={}
    Backtrack={}
    Pi[0]={}
    [Pi[0].update({st[n]:np.log10(0.5)+Em[x[0]][st[n]]}) for n in range(len(st))]
    for n in range(1,len(x)):
        Pi[n]={}
        [Pi[n].update({st[i]:max([Pi[n-1][st[j]]+Em[x[n]][st[i]]+Trans[st[i]][st[j]] for j in range(len(st))])})\
                       for i in range(len(st))]
        Backtrack[n]={}
        for i in range(len(st)):
            for j in range(len(st)):
                if Pi[n-1][st[j]]+Em[x[n]][st[i]]+Trans[st[i]][st[j]] == Pi[n][st[i]]:
                    Backtrack[n].update({st[i]:st[j]})
    pi_string=[]
    good_state=[k for k,v in Pi[len(x)-1].items() if v==max(Pi[len(x)-1].values())][0]
    pi_string=good_state
    for n in reversed(range(1,len(x))):
        print 10**Pi[n][good_state]
        new_state=Backtrack[n][good_state]
        pi_string+=new_state
        good_state=new_state
    return pi_string[::-1]

file='input.txt'
#file='input2.txt'
#file='../../Downloads/dataset_11632_10.txt'
#file='../../Downloads/rosalind_ba10i.txt'

f=open(file,'r')
lines=f.read().splitlines()
f.close()

x=list(lines[0])
alph=map(str,lines[2].split('\t'))
states=map(str,lines[4].split('\t'))
#print lines[8:]
footer=[len(lines)-n for n in reversed(range(len(lines))) if lines[n][0]=='-'][0]
Trans=pd.read_csv(file, skiprows=6,sep='\t+',header=0,index_col=0,skipfooter=footer)
Em=pd.read_csv(file, skiprows=len(lines)-footer+1,sep='\t+',header=0,index_col=0)
print 'x=',x
print 'alph=',alph
print 'states=',states

print Em
#print Em['A']['A']
print Trans

st= FindJiddenPath(x,alph,states,Trans,Em)
Em=np.log10(Em)
Trans=np.log10(Trans)

Fw={}

Fw[0]={}
[Fw[0].update({st[n]:np.log10(0.5)+Em[x[0]][st[n]]}) for n in range(len(st))]


for n in range(1,len(x)):
    Fw[n]={}
    [Fw[n].update({st[i]:np.log10(sum([10**(Fw[n-1][st[j]]+Em[x[n]][st[i]]+Trans[st[i]][st[j]]) for j in range(len(st))]))})\
                   for i in range(len(st))]
print Fw
Prx=sum([10**val for val in Fw[len(x)-1].values()])
print Prx



x= ['z', 'y', 'x', 'x', 'x', 'x', 'y', 'x', 'z', 'z']
alph= ['x', 'y', 'z']
states= ['A', 'B']
       x      y      z
A  0.356  0.191  0.453
B  0.040  0.467  0.493
       A      B
A  0.911  0.089
B  0.228  0.772
4.19031553963e-06
1.01538360912e-05
2.46044447947e-05
7.58656520021e-05
0.000436006988477
0.00134438938713
0.00414530700654
0.0127816913336
0.0394112265
{0: {'A': -0.64493179365114939}, 1: {'A': -0.40438004943042344}, 2: {'A': 0.10658832551545008}, 3: {'A': 0.6175567004613236}, 4: {'A': 1.1285250754071972}, 5: {'A': 1.6394934503530709}, 6: {'A': 1.8800451945737966}, 7: {'A': 2.3910135695196701}, 8: {'A': 3.0066301485055003}, 9: {'A': 3.6222467274913304}}
4190.31553963


/Users/juniper_admin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:60: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support skipfooter; you can avoid this warning by specifying engine='python'.
/Users/juniper_admin/anaconda/lib/python2.7/site-packages/IPython/kernel/__main__.py:61: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [7]:
''' Implement Baum-Welch Learning'''

' Implement Baum-Welch Learning'